# Analyse prédictive des accidents de la route
Ce notebook présente les étapes de prétraitement des données, l'équilibrage des classes avec SMOTE, et l'entraînement de trois modèles : Régression Logistique, Random Forest et XGBoost.

## 1. Chargement des bibliothèques

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')

## 2. Prétraitement des données
Supposons que le jeu de données soit déjà chargé dans `df_clean` et que la variable cible soit `gravite`. On supprime les colonnes trop révélatrices comme `nombre_deces`.

In [ ]:
colonnes_a_supprimer = ["date", "region", "nombre_deces", "gravite"]
X = df_clean.drop(columns=colonnes_a_supprimer)
y = df_clean["gravite"]

## 3. Rééquilibrage des classes avec SMOTE

In [ ]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

## 4. Séparation des données en train/test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

## 5. Entraînement et évaluation des modèles
On applique successivement la Régression Logistique, la Forêt Aléatoire et XGBoost.

In [ ]:
# Régression Logistique
print("=== Régression Logistique ===")
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
print(classification_report(y_test, y_pred_lr))

In [ ]:
# Random Forest
print("=== Random Forest ===")
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

In [ ]:
# XGBoost
print("=== XGBoost ===")
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

## 6. Analyse de Correlation 
La matrice de corrélation permet de visualiser les relations linéaires entre les variables explicatives.
Les couleurs indiquent la force et le sens de la corrélation : le rouge foncé traduit une corrélation positive, tandis que le bleu foncé signale une corrélation négative.
Cela permet d’identifier les variables fortement liées, comme l’âge et l’expérience du conducteur.

In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
sns.heatmap(df_clean.corr(numeric_only=True), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matrice de corrélation')
plt.show()


## 7. Deux visualisations SHAP (summary plot et force plot)
- Le SHAP summary plot présente les variables les plus influentes dans les prédictions du modèle.
Chaque ligne correspond à une variable, triée selon son importance, et chaque point représente une observation individuelle.
Les points rouges indiquent des valeurs élevées qui tendent à augmenter le risque, tandis que les bleus signalent des valeurs faibles qui peuvent réduire ce risque.

- Le SHAP force plot permet d’expliquer une prédiction spécifique en visualisant la contribution exacte de chaque variable.
En rouge figurent les facteurs qui augmentent le risque d’accident grave, et en bleu ceux qui le réduisent.
La somme de ces contributions justifie la probabilité finale attribuée par le modèle pour ce cas particulier.

In [ ]:
import shap

explainer = shap.Explainer(model_rf, X_train)
shap_values = explainer(X_test)

shap.summary_plot(shap_values, X_test, plot_type="dot")


In [ ]:

shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1][0], X_test.iloc[0], matplotlib=True)


In [ ]:

# SHAP Summary Plot
import shap
import numpy as np

explainer = shap.Explainer(rf, X_test)
shap_values = explainer(X_test)

shap.summary_plot(shap_values, X_test)


In [ ]:

# SHAP Force Plot pour une observation individuelle
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1][0], X_test.iloc[0])
